In [5]:
import anndata
import umap
import xgboost
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import pickle
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import display, Image, HTML
import mlflow

genes = ["CTNNB1", "JUND", "CD320", "CTNNB1", "IFI6"]
levels = ["0", "0.5", "1.0"]
data_dir = "/data/scgpt_perturbation_colon_epithel2/perturbations"
file_pattern = data_dir + "/perturbation_experiment_{gene}_level_{level}"
base_fname = file_pattern.format(gene = "JUND", level = "1.0")
base_embeddings = np.load(f"{base_fname}.embeddings.npy")
adata = anndata.read_h5ad(f"{base_fname}.h5ad")
base_df = adata.obs[["tissue", "cell_type", "disease"]].copy()
base_df.sample(5)

,tissue,cell_type,disease
cell_id,,,
N105446_L-ATTGTTCCAAACGTGG,lamina propria of mucosa of colon,paneth cell,Crohn disease
N105446_L-TCGACGGGTGAGACCA,lamina propria of mucosa of colon,paneth cell,Crohn disease
N105446_L-AGTAACCGTTAAGGGC,lamina propria of mucosa of colon,paneth cell,Crohn disease
N105446_L-GCAGGCTTCGCTAAAC,lamina propria of mucosa of colon,goblet cell,Crohn disease
N105446_L-ATCTTCATCTGAGAGG,lamina propria of mucosa of colon,goblet cell,Crohn disease


In [6]:
# can we train a model on top of the embeddings to classify disease state?    
X = base_embeddings
y = pd.Categorical(base_df["disease"]).codes
print(X.shape, y.shape)

(97788, 512) (97788,)


# ML experiment - compare models

In [7]:
import numpy as np
import pandas as pd
from typing import Dict, List
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, Lasso
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm




def evaluate_models(X: np.ndarray, y: np.ndarray, models: Dict[str, any], cv_splits: int) -> Dict[str, List[float]]:
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)
    results: Dict[str, List[float]] = {}
    
    for name, model in models.items():
        print(f"Evaluating {name}...")
        cv_scores: List[float] = []
        for train_index, test_index in tqdm(kf.split(X), total=kf.get_n_splits(), desc=f"CV Folds for {name}"):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred = y_pred.astype(int)
            score = accuracy_score(y_test, y_pred)
            cv_scores.append(score)
        results[name] = cv_scores
        print(f"{name}: Mean Accuracy: {np.mean(cv_scores):.3f}, Standard Deviation: {np.std(cv_scores):.3f}")
    return results

# Main setup
models = {
    'XGBoost': XGBClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000)
}

# Evaluate models
mlflow.autolog()
results = evaluate_models(X, y, models, cv_splits=5)
results

2024/05/02 08:44:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2024/05/02 08:44:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Evaluating XGBoost...


CV Folds for XGBoost:   0%|          | 0/5 [00:00<?, ?it/s]2024/05/02 08:44:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a2118ad4e0574c32897c444f1792efd7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2024/05/02 08:44:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:44:46] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/02 08:44:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
CV Folds for XGBoost:  20%|██        | 1/5 [00:30<02:02, 30.75s/it]2024/

XGBoost: Mean Accuracy: 0.840, Standard Deviation: 0.003
Evaluating Logistic Regression...


CV Folds for Logistic Regression: 100%|██████████| 5/5 [05:28<00:00, 65.72s/it]

Logistic Regression: Mean Accuracy: 0.875, Standard Deviation: 0.001


{'XGBoost': [0.8392984967788117,
  0.8441047141834543,
  0.8396564065855404,
  0.8361711919005983,
  0.8406708595387841],
 'Logistic Regression': [0.8754985172308007,
  0.876776766540546,
  0.8753962572860211,
  0.8734979802628215,
  0.8755944163215217]}

In [11]:
# train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

2024/05/02 09:03:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a7f2500b6eba4da8a0b8b5721f7fea9b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


LogisticRegression(max_iter=1000)

In [16]:
xgb_model = XGBClassifier()
xgb_model.fit(X, y)

2024/05/02 09:08:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4f2cd75ef07c4c5da0fc6751a5d2a938', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2024/05/02 09:09:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:09:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [18]:
# get the feature importnce from the xgboost model:
feature_importance = xgb_model.feature_importances_

# get the top 50 features indeces..
top_50_features = np.argsort(feature_importance)[::-1][:50]
top_50_features

array([367, 178, 110, 207, 450, 133, 457, 483, 449, 290, 385, 276, 155,
        77, 209, 252, 460, 266, 208, 375,  81, 272, 324, 470, 233, 401,
       108, 114, 458,  39, 204, 436, 441, 154, 329, 215, 248, 113, 247,
       238, 388,  67, 495, 135, 177, 371,  40, 330, 411, 382])

In [19]:
# test the model with the top 50 features:
X_top_50 = X[:, top_50_features]
results = evaluate_models(X_top_50, y, models, cv_splits=5)
results

Evaluating XGBoost...


CV Folds for XGBoost:   0%|          | 0/5 [00:00<?, ?it/s]2024/05/02 09:10:31 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fd179d9fc96e41a59ef84bc7da42dd51', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2024/05/02 09:10:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:10:35] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
CV Folds for XGBoost:  20%|██        | 1/5 [00:10<00:43, 10.82s/it]2024/05/02 09:10:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0be68062f967445caaa1a8698d2cdaf1', which will track hyperparameters, performance metrics, model artifacts, and lineage information f

XGBoost: Mean Accuracy: 0.824, Standard Deviation: 0.003
Evaluating Logistic Regression...


CV Folds for Logistic Regression: 100%|██████████| 5/5 [00:56<00:00, 11.25s/it]

Logistic Regression: Mean Accuracy: 0.807, Standard Deviation: 0.004


{'XGBoost': [0.827334083239595,
  0.8236015952551385,
  0.8187953778504959,
  0.8215984046632919,
  0.8268139285166437],
 'Logistic Regression': [0.8129665609980571,
  0.8086205133449228,
  0.8046835054709071,
  0.801912358746229,
  0.8089175231374955]}

In [ ]:
# save the xgboost model:
with open("xgboost_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)